# Air Quality: Design (wrap up) and Implement your Product

Welcome to the final lab of this project. Here again, you'll be working with the dataset you've now become familiar with from the air quality monitoring network in Bogotá [RMCAB](http://201.245.192.252:81/home/map). In this notebook, you will complete the following steps:

1. Import Python packages
2. Load the dataset with missing values filled in (output from the last lab)
3. Use the nearest neighbor method to make a map of PM2.5 in Bogotá
4. Test different values of k for the nearest neighbor method
5. Use the best value of k to make a map of PM2.5 in Bogotá
6. Construct a map animation of PM2.5 in Bogotá
7. Display your map animation

## 1. Importar Pacotes Python  

Execute a próxima célula para importar os pacotes Python necessários para este laboratório.  

Observe a linha `import utils`. Ela importa funções desenvolvidas especificamente para este laboratório. Se desejar visualizar essas funções, acesse `Arquivo -> Abrir...` e abra o arquivo `utils.py`.  

In [3]:
# Import Python packages.
import folium # package for animations
import folium.plugins as plugins # extras for animations
import pandas as pd # package for reading in and manipulating data
from sklearn.neighbors import KNeighborsRegressor # package for doing KNN
from datetime import datetime # package for manipulating dates

import utils1 # utility functions defined for this lab

print("All packages imported successfully!")

All packages imported successfully!


## 2. Carregar o conjunto de dados com valores faltantes preenchidos (resultado do laboratório anterior)  

Execute a próxima célula para ler o conjunto de dados que foi o resultado final do laboratório anterior - um dataset com todos os valores ausentes de poluentes devidamente preenchidos.

In [5]:
# Load the dataset with missing values filled in.
full_dataset = pd.read_csv('data/full_data_with_imputed_values.csv')
full_dataset['DateTime'] = pd.to_datetime(full_dataset['DateTime'], dayfirst=True)

full_dataset.head(5)

,DateTime,Station,Latitude,Longitude,PM2.5,PM10,NO,NO2,NOX,CO,OZONE,PM2.5_imputed_flag,PM10_imputed_flag,NO_imputed_flag,NO2_imputed_flag,NOX_imputed_flag,CO_imputed_flag,OZONE_imputed_flag
0,2021-01-01 00:00:00,USM,4.532097,-74.116947,32.7,56.6,7.504,15.962,23.493,0.44924,2.431,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-01 01:00:00,USM,4.532097,-74.116947,39.3,59.3,16.560,17.866,34.426,0.69832,1.121,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-01 02:00:00,USM,4.532097,-74.116947,70.8,96.4,22.989,17.802,40.791,0.88243,1.172,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-01 03:00:00,USM,4.532097,-74.116947,81.0,108.3,3.704,9.886,13.591,0.29549,6.565,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-01 04:00:00,USM,4.532097,-74.116947,56.1,87.7,2.098,9.272,11.371,0.16621,9.513,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 3. Utilizar o método do vizinho mais próximo para criar um mapa de PM2.5 em Bogotá  

Aqui você usará o método do vizinho mais próximo para estimar os valores dos poluentes nos pontos entre as estações, permitindo criar uma visualização espacial detalhada da poluição na cidade.

In [ ]:
# Define a value for k
k = 3
# Define the target pollutant
target = 'PM2.5'
# Define a grid cell size (higher value implies a finer grid)
n_points_grid = 64
neighbors_model = KNeighborsRegressor(n_neighbors=k, weights = 'distance', metric='sqeuclidean')
# Isolate a single time step from the dataset
time_step = datetime.fromisoformat('2021-04-05T08:00:00')
time_step_data = full_dataset[full_dataset['DateTime'] == time_step]
neighbors_model.fit(time_step_data[['Latitude', 'Longitude']], time_step_data[[target]])
# Generate a map of predictions for Bogotá
predictions_xy, dlat, dlon = utils.predict_on_bogota(neighbors_model, n_points_grid)
utils.create_heat_map(predictions_xy, time_step_data, dlat, dlon, target)

## 4. Testar diferentes valores de k para o método do vizinho mais próximo

Execute as células abaixo para:
1. Primeiro calcular o Erro Absoluto Médio (MAE) para k=1 (usando apenas um vizinho mais próximo, como foi feito no mapa acima)
2. Em seguida, realizar o mesmo cálculo para diferentes valores de k

Esta abordagem é similar à do laboratório anterior, onde você calculou o MAE usando medições de estações vizinhas para estimar os valores de PM2.5 em cada localização de estação. Aqui, vamos:

- Avaliar o método mostrado no mapa em cada localização de estação
- Simular a substituição da medição real da estação por:
  * Um valor da estação vizinha mais próxima (k=1)
  * Uma média ponderada das k estações vizinhas mais próximas (k>1)

O cálculo do Erro Absoluto Médio (MAE) realizado pelo código segue a fórmula:

$$MAE = \frac{1}{n} \sum_{i=1}^{n}{|\rm{real}_i - \rm{modelo}_i|}$$

Onde:
- "n" = número de amostras no conjunto de teste
- "real_i" = valor medido na estação
- "modelo_i" = valor estimado pelo método

In [ ]:
# Make an estimate of mean absolute error for k=1
utils.calculate_mae_for_k(full_dataset, k=1, target_pollutant=target)

Após testar k=1, execute a próxima célula para testar uma variedade de valores para k.

In [ ]:
# Make an estimate of mean absolute error (MAE) for a range of k values.
kmin = 1
kmax = 7

for kneighbors in range(kmin, kmax+1):
    mae = utils.calculate_mae_for_k(full_dataset, k=kneighbors, target_pollutant=target)
    print(f'k = {kneighbors}, MAE = {mae}')

## 5. Utilizar o melhor valor de k para criar um mapa de PM2.5 em Bogotá

Execute a célula abaixo para gerar um mapa de valores de PM2.5. O mapa mostrará a concentração do poluente escolhido na cidade na data selecionada (`end_date`). Ao clicar nos círculos do mapa (estações), serão exibidos gráficos pop-up mostrando a concentração do poluente no intervalo de tempo selecionado (de `start_date` até `end_date`). Você pode alterar os valores das datas, horários ou `k` para ver como os dados variam em diferentes períodos e como o resultado muda de acordo com `k`.

In [ ]:
k = 3
start_date = datetime.fromisoformat('2021-08-02T08:00:00')
end_date = datetime.fromisoformat('2021-08-05T08:00:00')

utils.create_heat_map_with_date_range(full_dataset, start_date, end_date, k, target)

## 6. Construir uma animação do mapa de PM2.5 em Bogotá

Execute a próxima célula para gerar uma animação da variação de PM2.5 em um intervalo de tempo específico. Você pode alterar o valor de k para usar um número diferente de vizinhos e modificar as datas e horários para analisar um período diferente.

In [ ]:
# Choose parameters for the animation
k = 3
n_points_grid = 64
# Filter a date range
start_date = datetime.fromisoformat('2021-08-04T08:00:00')
end_date = datetime.fromisoformat('2021-08-05T08:00:00')

# Create the features for the animation (these are the shapes that will appear on the map)
features = utils.create_animation_features(full_dataset, start_date, end_date, k, n_points_grid, target)
print('Features for the animation created successfully! Run the next cell to see the result!')

## 7. Mostre a animação do seu mapa

Rode a próxima célula para exibir a animação que você criou.

In [ ]:
# Create the map animation using the folium library
map_animation = folium.Map(location=[4.7110, -74.0721], zoom_start = 11) 
# Add the features to the animation
plugins.TimestampedGeoJson(
    {"type": "FeatureCollection", "features": features},
    period="PT1H",
    duration='PT1H',
    add_last_point=True
).add_to(map_animation)

# Run the animation
map_animation

## **Congratulations on finishing this lab!**

**Keep up the good work :)**